In [286]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import random
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
import plotly.express as px
from tkinter import *
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import re
import pandas as pd
import collections
import pickle
from itertools import combinations
import requests
from bs4 import BeautifulSoup
from fractions import Fraction
import statistics
pd.set_option('display.max_columns', None)


In [3]:
fot_cols = ['1 Accurate long balls', '1 Corners', '1 Dribbles attempted',
       '1 Dribbles succeeded', '1 Fouls conceded', '1 Goals',
       '1 Interceptions', '1 Long balls', '1 Offsides', '1 Pass success',
       '1 Possession', '1 Red cards', '1 Shots', '1 Blocked shots',
       '1 Shots woodwork', '1 Tackles attempted', '1 Throws', '1 Passes',
       '1 Shots on target', '1 Expected goals (xG)', '1 Yellow cards',
 '2 Accurate long balls', '2 Corners', '2 Dribbles attempted',
       '2 Dribbles succeeded', '2 Fouls conceded', '2 Goals',
       '2 Interceptions', '2 Long balls', '2 Offsides', '2 Pass success',
       '2 Possession', '2 Red cards', '2 Shots', '2 Blocked shots',
       '2 Shots woodwork', '2 Tackles attempted', '2 Throws', '2 Passes',
       '2 Shots on target', '2 Expected goals (xG)', '2 Yellow cards']

In [28]:
PATH = "C:\Program Files (x86)\chromedriver_win32\chromedriver.exe"
s=Service(PATH)
lod = []
loo = []

In [5]:
df = pd.read_csv('newstats.csv', index_col = 0)
features = df.iloc[: , :-2]
target = df.winner
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state = 3)
model = LogisticRegression(max_iter=100000, solver = 'lbfgs')#, class_weight = {0:0.275,1:0.462,2:0.263}, solver = 'lbfgs')
model.fit(X=X_train, y = y_train)
predicted = model.predict(X=X_test)
odds = model.predict_proba(X = X_test)
def hwin(s1, s2):
    if s1 > s2: 
        return 1
    elif s2 > s1:
        return 2
    else:
        return 0
X_test['fh winner'] = X_test.apply(lambda row: hwin(row['1 Goals'], row['2 Goals']), axis = 1)
X_test['winner'] = y_test
X_test['predicted'] = predicted
oddf = pd.DataFrame(odds, columns = ['% Tie', '% 1 Win', '% 2 Win'], index = X_test.index)
X_test = pd.concat([X_test, oddf], axis=1)

In [6]:
def kelly(p, b):
    return round((p*(b-1) - (1-p))/(b-1), 3)

In [7]:
def ebgs(url):
    driver = webdriver.Chrome(service=s)
    driver.get(url)
    time.sleep(random.randint(0, 4))
    mainstats = driver.find_elements(By.XPATH, '(//ul[@class="css-45dl0h-StatGroupContainer e683amr3"])')[0].text
    possession = driver.find_elements(By.XPATH, '(//div[@class="css-iwgu3j-PossessionWheel e683amr7"])')[0].text
    allstats = driver.find_elements(By.XPATH , '(//button[@class="css-1isl7sd-ExpandButton e7i6a0l1"])[2]')[0]
    allstats.click()
    time.sleep(random.randint(0, 6))
    stats = driver.find_elements(By.CLASS_NAME, 'e360fsv1')[1].text
    game = driver.find_elements(By.CLASS_NAME, 'css-jkaqxa')[1].text
    times = driver.find_elements(By.CLASS_NAME, 'css-xj92le')[1].text
    driver.quit()
    return mainstats, possession, stats, game, times

In [297]:
def getstats(url):
    mainstats, possession, stats, game, times = ebgs(url)
    pos = possession.split()[-2:]
    pos1, pos2 = pos[0], pos[1]
    score = game.split()[::2]
    goal1, goal2 = score[0], score[1]
    score = game.split()[::2]
    goal1 = score[0]
    goal2 = score[1]
    ms = re.split(pattern = "\\r?\\n", string = mainstats)
    os = ms[6:12]
    os.extend(ms[-9:])
    los = re.split(pattern = "\\r?\\n", string = stats)
    los.remove('Keeper')
    los.remove('Discipline')
    los.extend(os)
    try:
        los.remove('What is xG?')
    except:
        pass
    columns = los[1::3]
    columns.insert(0, 'Goals')
    columns.insert(2, 'Possession')
    team1 = los[::3]
    team1.insert(0, goal1)
    team1.insert(2, pos1)
    team2 = los[2::3]
    team2.insert(0, goal2)
    team2.insert(2, pos2)
    cols = ['1 ' + word for word in columns] + ['2 ' + word for word in columns]
    gs = team1 + team2
    row = {k:v for k, v in zip(cols, gs)}
    name = url.split('/')[-1]
    row['name'] = name
    lod.append(row)
    print(len(row))
    return row

In [252]:
def getodds(url):
    driver = webdriver.Chrome(service=s)
    driver.get(url)
    mainstats = driver.find_elements(By.XPATH, '//div[@class="defaultAnimation_dbigzdp"]')[0].text
    teams = re.split('vs| -', driver.find_elements(By.XPATH, '//div[@id="subevent-header"]')[0].text)[:2]
#     teams = driver.find_elements(By.XPATH, '//div[@id="subevent-header"]')[0].text.split(' ')[0:3:2]
    driver.quit()
    odds = [[int(i) for i in x.split('/')][0]/[int(i) for i in x.split('/')][1] + 1 for x in [mainstats.split('\n')[1::2][i] for i in [1, 0 , 2]]]
    print(odds)
    print(teams)
    loo.append(odds + teams)
    return odds

In [262]:
def getodds2(url):
    driver = webdriver.Chrome(service=s)
    driver.get(url)
    odde = driver.find_elements(By.XPATH, '//tbody[@id="t1"]')
    rawodds = [odde[i].text for i in range(len(odde))]
    driver.quit()
    ploop = rawodds[0].split('\n')
    oddict = {}
    for i in range(0, len(ploop), 2):
        fodds = ploop[i+1].split(' ')
        dodds = statistics.median([float(Fraction(j)) + 1.0 for j in fodds])
        oddict[ploop[i].lower()] = dodds
    hda = [txt.replace('-', ' ') for txt in url.split("/")[-2].split('-v-')]
    hda.insert(1, 'draw')
    odds = {k:oddict[k] for k in hda}
    odds = [list(odds.values())[l] for l in [1, 0, 2]]
    print(odds)
    return odds

In [263]:
def pred(tp, dha):
    tp = [tp[i] for i in fot_cols]
    tp = [float(x.strip('%'))/100 if '%' in x else float(x) for x in tp]
    fm = np.array(tp).reshape(1,-1)
    pp = model.predict_proba(fm)[0]
#     dubchance = [round(sum(x),4) for x in list(combinations(pp, 2))]
    print(f'\033[1mwin market:\033[0m\n  \tpred prob\t\tbook prob \tsize')
    for p, o, win in zip(pp, dha, ['draw', 'home','away']):
        bookp = round(1/o, 3)
        print(f'{win}:\t{round(p,3)}\t\t\t{bookp}\t\t{kelly(p, o) if p > bookp else ""}')
#     for col, stat in zip(fot_cols, tp):
#         print(col, stat)
#     print(f'\n\033[1mdouble chance:\033[0m\n  \tpredicted prob\t\tbook prob \tsize')
#     for p, o, win in zip(dubchance, dc, ['hwd', 'awd', 'hoaw']):
#         bookp = round(1/o, 3)
#         print(f'{win}:\t{round(p,3)}\t\t\t{bookp}\t\t{kelly(p, o) if p > bookp else ""}')

In [298]:
fotlink = "https://www.fotmob.com/match/3624515/matchfacts/hertha-bsc-vs-bayern-munchen"
oddslink = "https://www.oddschecker.com/football/germany/bundesliga/hertha-berlin-v-bayern-munich/winner"

halfstats = getstats(fotlink)
try:
    odds = getodds(oddslink)
except:
    odds = getodds2(oddslink)

89
[70.0, 401.0, 1.19]
['Hertha Berlin ', ' Bayern Munich']


In [299]:
pred(halfstats, odds)

win market:
  	pred prob		book prob 	size
draw:	0.017			0.014		0.003
home:	0.0			0.002		
away:	0.983			0.84		0.892


In [300]:
with open("test", "wb") as fp:
    pickle.dump((lod,loo), fp)
with open("test", "rb") as fp:
    lod, loo = pickle.load(fp)